In [ ]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import helper

import json
import os
import sys

In [ ]:
course_schedule = []
NO_WINDOW = False
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440

WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

RETRY_INTERVAL = 65

SIGIN_IN_EVERYTIME = False

In [ ]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

In [ ]:
def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [ ]:
def create_new_driver():
    #setup option for chrome profile
    chrome_options = Options()
    
    ## NEVER USE THE FOLLOWING OPTION SINCE IT WILL MAKE SUPPOSEDLY DIFFERENT WINDOWS FOR DIFFERENT CLASS MIXED IN SOME WAY!!!!
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    ## the following option is to solve the error if using the above cause a problem
    # chrome_options.add_argument("--remote-debugging-port=9222") 
    
    prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", prefs)

    if NO_WINDOW:
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-gpu')
        print("LOG: NO WINDOW")
    
    #start web driver
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)
    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    
    return driver, wait


In [ ]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element("id",'ctl00_cphContents_txtUsername')
    password = driver.find_element("id",'ctl00_cphContents_txtPassword')
    
    send_keys(username, "DN20092360")
    send_keys(password, "jj1216")
#     username.send_keys("DN23035242")
#     password.send_keys("NTW3BSFH")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))

    sign_in_btn = driver.find_element(By.CLASS_NAME,"submit-button")
    sign_in_btn.click()

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class():
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), '預定課')]")
#     print("click_btn: ", click_btn)
    click_btn.click()
    
def click_book_this_class_now(driver, wait):

    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), '現在就預定此課')]")

    if click_btn:
        click_btn.click()
        return click_btn


In [ ]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [ ]:
temp = []

# load schedule from file
with open("./class_schedules/06 十一月 - 12 十一月 2023.json", "r") as read_file:
    course_schedule = json.load(read_file)

Day_string = "10"

c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Aeri', c)
c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('pm', c)

temp.extend(c)

courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

In [ ]:
master_counter = 1

In [ ]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [ ]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [ ]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [ ]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [ ]:
def reserve_class(driver, wait, course_to_be_booked, idx):
    
    global master_counter
    global class_idx
    global num_classes

    stride =RETRY_INTERVAL / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    

    driver.get(book_url)

    click_btn=click_book_this_class_now(driver, wait)
    if click_btn is not None:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=RETRY_INTERVAL)
        log_str = "'{}','{}' by '{}',FULL,'{}'\nnext_book_datetime={}\nclick_btn={}\n".format(master_counter, class_name, teacher, datetime.datetime.now(), next_book_datetime, click_btn)
        print(log_str)
        
        if 0==0: #(master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
       
        if SIGIN_IN_EVERYTIME:
            driver.close()
            
            driver, wait = create_new_driver()
            driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
            driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

            sign_in_once(driver, wait)

        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
        print("now={}".format(datetime.datetime.now()))
        print("next_book_datetime={}".format(next_book_datetime))
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [ ]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
    print("now={}".format(datetime.datetime.now()))
    print("next_book_datetime={}".format(next_book_datetime))
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()